In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
# import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Premotor table

In [ ]:
pre_to_df = connectome_create.load_pre_to_df(ext='ordered_no_frags')

In [ ]:
pre_to_df

In [ ]:
pre_to_mn_df = connectome_create.load_old_pre_to_mn_df(ext='matched_typed_with_nt')

In [ ]:
pre_to_mn_df.groupby(level=[2,3]).size()

In [ ]:
pre_to_df

# How motor-y are different pMNs?

## Local vnc neurons
This cell plots the outputs of identified local neurons. It counts the number of total synapses, it computes the fraction onto motor neurons. Then it computes the fraction onto other classes and plots the total fraction.

In [ ]:
print(pre_to_df.index.get_level_values('cell_class').unique())

classlist = ['motor', 
                    'descending', 
                    'sensory', 
                    'ascending', 
                    'intersegmental',
                    'local', 
                    'non_t1_motor', 
                    'descending_post', 
                    'sensory_post',
                    'ascending_post', 
                    'neurmodulatory_ascending_neuron',
                    'vnc_non_premotor']

In [ ]:
# How many local neurons? # ~536
# segID	motor	has_soma	sensory	 neck	local	
All = slice(None)
local = pre_to_df.loc[:,('local',All,All,All,All)]
local.head()

# cell_class	preferred_pool	NT	classification_system	cell_type



In [ ]:
# How many intersegmental neurons? # ~350
# segID	motor	has_soma	sensory	 neck	local	

def get_motory_fraction(pre_df,c_tup,compress_other_classes=False,srtstr='motor'):
    ct_df = pre_df.loc[:,c_tup] # a more generalized version of the above

    # pd.Series

    # pd.DataFrame of fractional inputs
    cell_classes = ct_df.index.get_level_values('cell_class').unique()
    frac_df = pd.DataFrame(index=cell_classes,columns=ct_df.columns)
    for key in cell_classes:
        frac_df.loc[key,:] = ct_df.loc[key,:].sum(axis=0)        

    ct_to_all    = ct_df.loc[All,:].sum(axis=0)                         # calculate the total
    frac_df = frac_df/ct_to_all                                         # divide by the total
    x = [i for i in range(len(ct_df.loc['motor',:].sum(axis=0)))]   
    frac_df.loc['x',:] = x                                              # this x is the original order of the cell_class
    frac_df.loc['total',:] = ct_to_all

    if compress_other_classes:
        otherlist = ['non_t1_motor', 
                    'descending_post', 
                    'sensory_post',
                    'ascending_post', 
                    'neurmodulatory_ascending_neuron',
                    'vnc_non_premotor']
        frac_df.loc['other',:] = frac_df.loc[otherlist,:].sum(axis=0)
        frac_df = frac_df.drop(otherlist,axis=0)
        
    frac_df = frac_df.sort_values([srtstr],axis=1,ascending=False)    # sort according to how t1 motory
    sorted_total=frac_df.loc['total',:].to_numpy()                      # keep the sorted total 
    frac_df = frac_df.drop('total',axis=0)                             # drop the total row
    
    return frac_df, sorted_total                                       



In [ ]:
def get_cum_motor_premotor(pre_df,c_tup):
    ct_df = pre_df.loc[:,c_tup] # a more generalized version of the above

    # pd.Series

    # pd.DataFrame of fractional inputs
    cell_classes = ct_df.index.get_level_values('cell_class').unique()
    frac_df = pd.DataFrame(index=cell_classes,columns=ct_df.columns)
    for key in cell_classes:
        frac_df.loc[key,:] = ct_df.loc[key,:].sum(axis=0)        

    ct_to_all    = ct_df.loc[All,:].sum(axis=0)                         # calculate the total
    x = [i for i in range(len(ct_df.loc['motor',:].sum(axis=0)))]   
    frac_df.loc['x',:] = x                                              # this x is the original order of the cell_class
    frac_df.loc['total',:] = ct_to_all

    otherpre = ['descending', 
                'sensory', 
                'ascending', 
                'intersegmental',]
                
    frac_df.loc['other_pre',:] = frac_df.loc[otherpre,:].sum(axis=0)


    otherlist = ['non_t1_motor', 
                'descending_post', 
                'sensory_post',
                'ascending_post', 
                'neurmodulatory_ascending_neuron',
                'vnc_non_premotor']
    
    frac_df.loc['other',:] = frac_df.loc[otherlist,:].sum(axis=0)
    frac_df.loc['other_frac',:] = frac_df.loc['other',:]/ct_to_all 
    frac_df = frac_df.drop(otherlist,axis=0)
    
    frac_df = frac_df.sort_values(['other_frac'],axis=1,ascending=True)    # sort according to how other fraction
    sorted_total=frac_df.loc['total',:].to_numpy()                      # keep the sorted total 
    frac_df = frac_df.drop('total',axis=0)                             # drop the total row

    frac_df.loc['cum_motor',:] = frac_df.loc['motor',:].cumsum()
    frac_df.loc['cum_motor_frac',:] = frac_df.loc['cum_motor',:]/frac_df.loc['motor',:].sum()

    frac_df.loc['cum_local',:]          = frac_df.loc['local',:].cumsum()
    frac_df.loc['cum_descending',:]     = frac_df.loc['descending',:].cumsum()
    frac_df.loc['cum_sensory',:]        = frac_df.loc['sensory',:].cumsum()
    frac_df.loc['cum_intersegmental',:] = frac_df.loc['intersegmental',:].cumsum()
    frac_df.loc['cum_other_pre',:] = frac_df.loc['other_pre',:].cumsum()
    

    # clean up a little
    frac_df.loc['cum_motor',frac_df.loc['cum_motor',:]==0] = np.nan
    frac_df.loc['cum_local',frac_df.loc['cum_local',:]==0] = np.nan
    frac_df.loc['cum_other_pre',frac_df.loc['cum_other_pre',:]==0] = np.nan

    # frac_df.loc['cum_motor_over_pre',:] = frac_df.loc['cum_motor',:]/frac_df.loc['cum_local',:]
    frac_df.loc['cum_local_over_motor',:] = frac_df.loc['cum_local',:]/frac_df.loc['cum_motor',:]
    frac_df.loc['cum_other_over_motor',:] = frac_df.loc['cum_other_pre',:]/frac_df.loc['cum_motor',:]

    # frac_df.loc['cum_local_over_motor',frac_df.loc['cum_local_over_motor',:]>10] = np.nan
    # frac_df.loc['cum_other_over_motor',frac_df.loc['cum_other_over_motor',:]>10] = np.nan

    return frac_df, sorted_total



In [ ]:
                                       
fig, ax = plt.subplots(3, 5, figsize=(24,10),gridspec_kw={'height_ratios': [1, 2, 3]})
xdim = pre_to_df.loc[:,('local',All,All,All,All)].shape[1]

cell_clses = ['descending','sensory','ascending','intersegmental','local']
ax_nums = [i for i in range(len(cell_clses))]
for cls,cnt in zip(cell_clses,ax_nums):
    # Descending
    # 0) plot the 'other' fraction in the top plot
    frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,(cls,All,All,All,All))
    x = [i for i in range(len(sorted_total))]
    ax[0,cnt].plot(x,frac_df.loc['other_frac',:],color='#000000')
    ax[0,cnt].set_xlim(0,xdim)
    ax[0,cnt].set_ylim(0,1.05)

    # 1) plot the cumulative motor fraction
    ax[1,cnt].plot(x,frac_df.loc['cum_motor',:],color='#910951')
    ax[1,cnt].set_xlim(0,xdim)
    

    # 2) plot the cumulative numbers 
    ax[2,cnt].plot(x,frac_df.loc['cum_local_over_motor',:],color="#7D688E",label='local')
    ax[2,cnt].plot(x,frac_df.loc['cum_descending',:]/frac_df.loc['cum_motor',:],color="#79D0F7",label='descending')
    ax[2,cnt].plot(x,frac_df.loc['cum_sensory',:]/frac_df.loc['cum_motor',:],color="#93E5B6",label='sensory')
    ax[2,cnt].plot(x,frac_df.loc['cum_intersegmental',:]/frac_df.loc['cum_motor',:],color="#CC9FCC",label='intersegmental')
    ax[2,cnt].plot(x,frac_df.loc['cum_other_pre',:]/frac_df.loc['cum_motor',:],color="#000000",label='non-local')
    ax[2,cnt].set_xlim(0,xdim)
    ax[2,cnt].set_ylim(0,5)

    ax[0,cnt].set_title(cls)
    ax[2,cnt].set_xlabel('N ' + cls + ' pMNs')

ax[0,0].set_ylabel('% to non-pMN')
ax[1,0].set_ylabel('cumu. MN syn.')
ax[2,0].set_ylabel('ratio of cum. clss to mn')

ax[2,0].legend()

fig.savefig('./figpanels/' + 'cumulative_motoryness' + '.svg',format='svg')

## Look at local outputs vs mn outputs for different classes

### 3D

In [ ]:
# fig, ax = plt.subplots( 1, 3, figsize=(24,8))
fig = plt.figure(figsize=(24,8))
ax = [fig.add_subplot(1, 3, 1, projection='3d'), fig.add_subplot(1, 3, 2, projection='3d'), fig.add_subplot(1, 3, 3, projection='3d')]


cell_clses = ['descending','intersegmental','local']
# cell_clses = ['descending']
ax_nums = [i for i in range(len(cell_clses))]
lim = [2000,2000,2000]

for cls,cnt in zip(cell_clses,ax_nums):
    # Descending
    # 0) plot the 'other' fraction in the top plot
    frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,(cls,All,All,All,All))
    x = [i for i in range(len(sorted_total))]
    # ax[cnt].plot([0, frac_df.loc['motor',:].max()],[0, frac_df.loc['motor',:].max()],color='#cccccc',)
    # ax[cnt].plot(frac_df.loc['motor',:],frac_df.loc['local',:],color='#000000',ls='None', marker='.')

    
    ax[cnt].scatter(frac_df.loc['motor',:],frac_df.loc['local',:],frac_df.loc['other',:],color='#000000',ls='None', marker='.')
    
    ax[cnt].set_xlim([0,lim[cnt]])
    ax[cnt].set_ylim([0,lim[cnt]])
    ax[cnt].set_zlim([0,lim[cnt]])
    # ax[cnt].set_aspect('equal')
    ax[cnt].set_xlabel('synapses onto motor')
    ax[cnt].set_ylabel('output onto local')
    ax[cnt].set_zlabel('output onto non-premotor (but has soma)')
    
    more_loc_than_motor = (frac_df.loc['local',:] > frac_df.loc['motor',:]).sum()/frac_df.loc['local',:].count()
    print(more_loc_than_motor)



In [ ]:
# Plot them all on the same plot with different colors
fig = plt.figure(figsize=(24,14))
ax = [fig.add_subplot(1, 1, 1, projection='3d')]

cell_clses = ['descending','intersegmental','local']
# cell_clses = ['descending']
ax_nums = [i for i in range(len(cell_clses))]
lim = [2000,2000,2000]

classlist = ['motor', 'descending', 'sensory', 'ascending', 'intersegmental','local',   'non_t1_motor', 'descending_post',  'sensory_post','ascending_post','neurmodulatory_ascending_neuron', 'vnc_non_premotor']
colors = [  "#000000",  "#CC9FCC",        "#7D688E", "#CC4893",      '#79D0F7',          "#93E5B6",      "#6084CC",      "#CC4893",                          "#CC9FCC"]
cmap = sns.set_palette(sns.color_palette(colors))

for cls,clr in zip(cell_clses,colors):
    # Descending
    # 0) plot the 'other' fraction in the top plot
    frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,(cls,All,All,All,All))
    x = [i for i in range(len(sorted_total))]
    # ax[cnt].plot([0, frac_df.loc['motor',:].max()],[0, frac_df.loc['motor',:].max()],color='#cccccc',)
    # ax[cnt].plot(frac_df.loc['motor',:],frac_df.loc['local',:],color='#000000',ls='None', marker='.')

    
    ax[0].scatter(frac_df.loc['motor',:],frac_df.loc['local',:],frac_df.loc['other',:],color=clr,ls='None', marker='.')
    
    ax[0].set_xlim([0,lim[cnt]])
    ax[0].set_ylim([0,lim[cnt]])
    ax[0].set_zlim([0,lim[cnt]])
    # ax[cnt].set_aspect('equal')
    ax[0].set_xlabel('synapses onto motor')
    ax[0].set_ylabel('output onto local')
    ax[0].set_zlabel('output onto other')
    more_loc_than_motor = (frac_df.loc['local',:] > frac_df.loc['motor',:]).sum()/frac_df.loc['local',:].count()
    print(more_loc_than_motor)



### 2D

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(24,8))
# fig = plt.figure(figsize=(24,8))
# ax = [fig.add_subplot(1, 3, 1, projection='3d'), fig.add_subplot(1, 3, 2, projection='3d'), fig.add_subplot(1, 3, 3, projection='3d')]


cell_clses = ['descending','intersegmental','local']
# cell_clses = ['descending']
ax_nums = [i for i in range(len(cell_clses))]
lim = [4000,6000,4000]

for cls,cnt in zip(cell_clses,ax_nums):
    # Descending
    # 0) plot the 'other' fraction in the top plot
    frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,(cls,All,All,All,All))
    x = [i for i in range(len(sorted_total))]
    ax[cnt].plot([0, frac_df.loc['motor',:].max()],[0, frac_df.loc['motor',:].max()],color='#cccccc',)
    ax[cnt].plot(frac_df.loc['motor',:],frac_df.loc['local',:],color='#000000',ls='None', marker='.')

    
    ax[cnt].scatter(frac_df.loc['motor',:],frac_df.loc['local',:],color='#000000',ls='None', marker='.')
    
    ax[cnt].set_xlim([0,lim[cnt]])
    ax[cnt].set_ylim([0,lim[cnt]])
    ax[cnt].set_aspect('equal')
    
    ax[cnt].set_xlabel('synapses onto motor')
    ax[cnt].set_ylabel('synapses onto local')
    
    more_loc_than_motor = (frac_df.loc['local',:] > frac_df.loc['motor',:]).sum()/frac_df.loc['local',:].count()
    print(more_loc_than_motor)
    print(frac_df.loc['local',:].sum()/frac_df.loc['motor',:].sum() )


### Pie chart  

In [ ]:
def get_sums_for_pie(pre_df,c_tup,condense_pre=False,condense_post=False):
    ct_df = pre_df.loc[:,c_tup] # a more generalized version of the above

    cell_classes = ct_df.index.get_level_values('cell_class').unique()
    frac_df = pd.DataFrame(index=cell_classes,columns=ct_df.columns)
    for key in cell_classes:
        frac_df.loc[key,:] = ct_df.loc[key,:].sum(axis=0)        

    if condense_pre:
        otherpre = ['descending', 
                'sensory', 
                'ascending', 
                'intersegmental',]
    
        frac_df.loc['other_pre',:] = frac_df.loc[otherpre,:].sum(axis=0)
        frac_df = frac_df.drop(otherpre,axis=0)

    if condense_post:
        otherlist = ['non_t1_motor', 
                'descending_post', 
                'sensory_post',
                'ascending_post', 
                'neurmodulatory_ascending_neuron',
                'vnc_non_premotor']
    
        frac_df.loc['other',:] = frac_df.loc[otherlist,:].sum(axis=0)
        frac_df = frac_df.drop(otherlist,axis=0)

    return frac_df



In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(24,8))
# fig = plt.figure(figsize=(24,8))
# ax = [fig.add_subplot(1, 3, 1, projection='3d'), fig.add_subplot(1, 3, 2, projection='3d'), fig.add_subplot(1, 3, 3, projection='3d')]


cell_clses = ['descending','intersegmental','local']
# cell_clses = ['descending']
ax_nums = [i for i in range(len(cell_clses))]
lim = [4000,6000,4000]

for cls,cnt in zip(cell_clses,ax_nums):
    pie_df = get_sums_for_pie(pre_to_df,(cls,All,All,All,All),condense_pre=False,condense_post=False)
    
    sizes = pie_df.sum(axis=1)

    ax[cnt].pie(sizes, explode=None, labels=sizes.index, autopct='%1.0f%%',
              shadow=False, startangle=90)
    ax[cnt].axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

#     plt.show()

fname = './figpanels/class_input_to_class_pie.svg'
fig.savefig(fname,format='svg')

In [ ]:
sizes

# Example neurons for figure

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()
def conn_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(np.log10(df.T.to_numpy()+1), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if figfilename is not None:
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

def lin_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(df.T.to_numpy(), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'linear', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if figfilename is not None:
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

def keep_first_string_in_list(in_list):
    out_list = []
    key = None
    for s in in_list:
        if key is None:
            key = s
            out_list = out_list + [key]
            continue
        elif s==key:
            out_list = out_list + ['']
        else:
            key = s
            out_list = out_list + [s]
    return out_list

## Descending neurons

In [ ]:
# Descending neuron that makes most of its connections to local neurons
frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,('descending',All,All,All,All))
print(frac_df.loc[:,frac_df.loc['local',:]>2000].columns.get_level_values('segID'))

print(frac_df.loc[:,frac_df.loc['motor',:]>1000].columns.get_level_values('segID'))



In [ ]:
print(frac_df.loc['motor',:].sum())
print(frac_df.loc['motor',frac_df.loc['motor',:]>1000].sum())

## Ascending neurons

In [ ]:
# fig, ax = plt.subplots( 1, 3, figsize=(24,8))
fig = plt.figure(figsize=(24,8))
ax = [fig.add_subplot(1, 1, 1, projection='3d')]


cell_clses = ['ascending']
ax_nums = [i for i in range(len(cell_clses))]
lim = [2000,2000,2000]

for cls,cnt in zip(cell_clses,ax_nums):
    # Descending
    # 0) plot the 'other' fraction in the top plot
    frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,(cls,All,All,All,All))
    x = [i for i in range(len(sorted_total))]
    # ax[cnt].plot([0, frac_df.loc['motor',:].max()],[0, frac_df.loc['motor',:].max()],color='#cccccc',)
    # ax[cnt].plot(frac_df.loc['motor',:],frac_df.loc['local',:],color='#000000',ls='None', marker='.')

    
    ax[cnt].scatter(frac_df.loc['motor',:],frac_df.loc['local',:],frac_df.loc['other',:],color='#000000',ls='None', marker='.')
    
    ax[cnt].set_xlim([0,lim[cnt]])
    ax[cnt].set_ylim([0,lim[cnt]])
    ax[cnt].set_zlim([0,lim[cnt]])
    # ax[cnt].set_aspect('equal')
    ax[cnt].set_xlabel('synapses onto motor')
    ax[cnt].set_ylabel('output onto local')
    ax[cnt].set_zlabel('output onto non-premotor (but has soma)')
    
    more_loc_than_motor = (frac_df.loc['local',:] > frac_df.loc['motor',:]).sum()/frac_df.loc['local',:].count()
    print(more_loc_than_motor)


In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(24,8))
# fig = plt.figure(figsize=(24,8))
# ax = [fig.add_subplot(1, 3, 1, projection='3d'), fig.add_subplot(1, 3, 2, projection='3d'), fig.add_subplot(1, 3, 3, projection='3d')]


cell_clses = ['ascending']
ax_nums = [i for i in range(len(cell_clses))]
lim = [4000,6000,4000]

for cls,cnt in zip(cell_clses,ax_nums):
    # Descending
    # 0) plot the 'other' fraction in the top plot
    frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,(cls,All,All,All,All))
    x = [i for i in range(len(sorted_total))]
    ax[cnt].plot([0, frac_df.loc['motor',:].max()],[0, frac_df.loc['motor',:].max()],color='#cccccc',)
    ax[cnt].plot(frac_df.loc['motor',:],frac_df.loc['local',:],color='#000000',ls='None', marker='.')

    
    ax[cnt].scatter(frac_df.loc['motor',:],frac_df.loc['local',:],color='#000000',ls='None', marker='.')
    
    ax[cnt].set_xlim([0,lim[cnt]])
    ax[cnt].set_ylim([0,lim[cnt]])
    ax[cnt].set_aspect('equal')
    
    ax[cnt].set_xlabel('synapses onto motor')
    ax[cnt].set_ylabel('synapses onto local')
    
    more_loc_than_motor = (frac_df.loc['local',:] > frac_df.loc['motor',:]).sum()/frac_df.loc['local',:].count()
    print(more_loc_than_motor)
    print(frac_df.loc['local',:].sum()/frac_df.loc['motor',:].sum() )


In [ ]:
# A neuron that makes most of its connections to local neurons
frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,('ascending',All,All,All,All))
print(frac_df.loc[:,frac_df.loc['local',:]>2000].columns.get_level_values('segID'))

print(frac_df.loc[:,frac_df.loc['motor',:]>1000].columns.get_level_values('segID'))


In [ ]:
648518346496495106
input_tup = ('ascending',['thorax_stance','thorax_swing'],All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

pre_df_slice = pre_to_df.loc[output_tup,input_tup]

xticklabels = keep_first_string_in_list(pre_df_slice.index.get_level_values('preferred_pool').to_list())
conn_heat_map(df=pre_df_slice, xticks=xticklabels,yticks =[i for i in range(pre_df_slice.shape[1])], figsz=[10,10])


## Intersegmental neurons

In [ ]:

# Intersegmental neuron that makes a lot of local connections
frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,('intersegmental',All,All,All,All))
print(frac_df.loc[:,frac_df.loc['local',:]>2000].columns.get_level_values('segID'))
frac_df.loc[:,frac_df.loc['other',:]>2000]
# Just moved this one over to the local neurons (8/18)

frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,('local',All,All,All,All))
print(frac_df.loc[:,frac_df.loc['local',:]>3500].columns.get_level_values('segID'))
# frac_df.loc[:,frac_df.loc['other',:]>2000]


### FETi contact intersegmental neurons

In [ ]:
# cell_class	preferred_pool	NT	classification_system	cell_type

input_tup = ('intersegmental','tibia_extensor',All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

pre_df_slice = pre_to_df.loc[output_tup,input_tup]

%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()

fig = plt.figure(1, figsize = [8,16])
ax = sns.heatmap(np.log10(pre_df_slice.T.to_numpy()+1), cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('All to motor', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()


In [ ]:
# Some of the extensor contacting MNs
pre_df_slice.iloc[:,7:11]

### The claw contacting intersegmental neurons

In [ ]:
# cell_class	preferred_pool	NT	classification_system	cell_type

input_tup = ('intersegmental','ltm',All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

pre_df_slice = pre_to_df.loc[output_tup,input_tup]

xticklabels = keep_first_string_in_list(pre_df_slice.index.get_level_values('preferred_pool').to_list())
conn_heat_map(df=pre_df_slice, xticks=xticklabels,yticks =[i for i in range(pre_df_slice.shape[1])], figsz=[10,10])


In [ ]:
# Some of the extensor contacting MNs
pre_df_slice.iloc[:,[0,1,3,5,6]]

## Local examples

### Femur reductor example

In [ ]:
# cell_class	preferred_pool	NT	classification_system	cell_type

input_tup = ('local','femur_reductor',All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

pre_df_slice = pre_to_df.loc[output_tup,input_tup]

xticklabels = keep_first_string_in_list(pre_df_slice.index.get_level_values('preferred_pool').to_list())
conn_heat_map(df=pre_df_slice, xticks=xticklabels,yticks =[i for i in range(pre_df_slice.shape[1])], figsz=[10,10])


In [ ]:
# Some of the extensor contacting MNs
pre_df_slice.iloc[:,[1,2,3]]

### R neurons

In [ ]:
# cell_class	preferred_pool	NT	classification_system	cell_type

input_tup = ('local',All,All,'R',All,All)
output_tup = ('motor',All,All,All,All,All)

pre_df_slice = pre_to_df.loc[output_tup,input_tup]

xticklabels = keep_first_string_in_list(pre_df_slice.index.get_level_values('preferred_pool').to_list())
conn_heat_map(df=pre_df_slice, xticks=xticklabels,yticks =[i for i in range(pre_df_slice.shape[1])], figsz=[10,10])


In [ ]:
# Some of the extensor contacting MNs
pre_df_slice.iloc[:,[0,2,3,8,13]]
pre_df_slice.columns.get_level_values('segID').to_list()
pre_df_slice

# Now that preferred pools and all have been added to the index, groupby HL and preferred pool

# Plot bar plots

In [ ]:

# # locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
          #'motor', 'nont1',    'desc',     'sens',     'ascend',   'local',    'intr',     'vnc',      'fragments'
classlist = ['motor', 'descending', 'sensory', 'ascending', 'intersegmental','local',   'non_t1_motor', 'descending_post',  'sensory_post','ascending_post','neurmodulatory_ascending_neuron', 'vnc_non_premotor']
colors = ["#910951",  "#79D0F7",    "#93E5B6",  "#6084CC",  "#CC9FCC",        "#7D688E", "#CC4893",      '#79D0F7',          "#93E5B6",      "#6084CC",      "#CC4893",                          "#CC9FCC"]
cmap = sns.set_palette(sns.color_palette(colors))

In [ ]:
# ax = frac_df.T.plot(x='x',kind='barh', stacked=True, legend = True,align='edge') 
local_target_frac_df, sorted_total = get_motory_fraction(pre_to_df,('local',All,All,All,All),compress_other_classes=True,srtstr='other')
ax = local_target_frac_df.T.plot(x='x',kind='barh', stacked=True, legend = True,width=1,cmap=cmap,figsize=(4,10)) 
# Most of them are pretty close to 1